In [10]:
# Shared setup
%run _shared/setup_data_overview.py

Python: 3.12.11 (main, Jul  1 2025, 05:29:09) [GCC 12.2.0]
Pandas: 2.3.3
Plotly: 6.5.0
Repo root: /workspaces/WjecDocumentScraper
Data dir: /workspaces/WjecDocumentScraper/notebooks/cleansed_data
Loaded issues: (24771, 12) files: (239, 3)


In [11]:
# Quick sanity checks
issues.shape, files.shape

((24771, 12), (239, 3))